In [ ]:
import sys
from PIL import Image
import pyocr
import pyocr.builders
import cv2
import datetime
import glob
import numpy as np
import os

In [ ]:
#Take a photo with the webcam and save it under the date+img_counter


#Obtain the current date and time with now and transform it to string with seq
# Concatenate the string to "img" in order to make image title
now = datetime.datetime.now()
seq = str(now.strftime("%Y%m%d%H%M"))
title = "img"+seq



cam = cv2.VideoCapture(0) #Capture image with Camera at location 0
cv2.namedWindow("Take Picture") #Frame title

img_counter = 0 #counter for image taken in the same date and/or timeframe
while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed (to stop camera)
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed (to take a shot)
        img_name = title + "_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1


cam.release()

cv2.destroyAllWindows()

In [ ]:
#Image transformation to gray scale and save to new folder

counter =0
for infile in glob.glob("./*.jpg"):     # Set path and open all .png images
    counter+=1
    print(counter)
    file, ext = os.path.splitext(infile)
    img_array = Image.open(infile)
    img_array = np.array(img_array)
    path = './grayscaled'

    image = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY) #graysacle images
    cv2.imwrite(os.path.join(path , file+'.jpg'), image) #save grayscaled images to new path as .png

print("----end----")

In [ ]:
## OCR program for word recognition
tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("No OCR tool found")
    sys.exit(1)

tool = tools[0]
print("Will use tool '%s'" % (tool.get_name()))


txt = tool.image_to_string(
    #文字認識対象の画像image.pngを用意する
    Image.open("./grayscaled/xyz.jpg"),
    lang="jpn",
    builder=pyocr.builders.TextBuilder(tesseract_layout=6)
)

print( txt )

#僕のフォルダ構成は下記の通りです。

#detectText/
#- detect.py
#- image.png 

In [ ]:
#Same as above but extract text from all images in the folder

tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("No OCR tool found")
    sys.exit(1)

tool = tools[0]
print("Will use tool '%s'" % (tool.get_name()))

cnt = 0 # for number of text extracted images
for infile in glob.glob("./*.jpg"):     # Set path and open all .png (or .jpg) images
    cnt+= 1
    file, ext = os.path.splitext(infile)
    img_array = Image.open(infile)
    print("IMAGE No",cnt,"=",file,"\n")

    txt = tool.image_to_string(     #pass image to text extractor
        img_array,
        lang="jpn",
        builder=pyocr.builders.TextBuilder(tesseract_layout=6)
    )
    print(txt) # print extracted text
    print("__________________________________________________________\n\n\n")